# 02 Stock Price Data Collection and Preprocessing
Now that I have the transcript data, I will need the corresponding stock price data to match. To obtain the dataset, I will use the yfinance API to obtain daily stock data. Furthermore, I will only obtain the price data between 2010 and 2024, as that matches the time frame of my transcripts.

# Imports

In [48]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import date
from tqdm import tqdm
import os
import warnings

# Matching Transcript Company Names to Tickers
Before obtaining the stock price data from yfinance, we first need to specify the tickers that we want to extract. Since the transcripts only have the full company name, I'll need to match each name to the tickers first.

## Reading in transcript data
This data was cleaned and preprocessed in the previous notebook. The purpose of reading this data in is to get a unique list of company names that we can match the tickers to.

In [46]:
# Reading in the full transcripts dataframe
merged_tscrpts_csv_path = '/Users/danielwang/Desktop/Berkeley MIDS Stuff/Berkeley MIDS Summer 2025 Stuff/Berkeley MIDS DATASCI 266/Berkeley MIDS DATASCI 266 Project Material/SNP500_Transcripts_2010_to_2024.csv'
merged_tscrpts = pd.read_csv(merged_tscrpts_csv_path, sep='|', index_col=0)
merged_tscrpts['Date'] = pd.to_datetime(merged_tscrpts['Date'])

merged_tscrpts

,Unique_transcript_id,Date,Company_name,Word_count,Text
0,7.111315e+07,2010-01-06 14:30:00,Monsanto Company,6628,"Greetings, and welcome to the First Quarter 20..."
1,8.235962e+07,2010-01-06 15:00:00,"Family Dollar Stores, LLC",1970,"Good morning, my name is Kim, and I will be yo..."
2,8.353603e+07,2010-01-06 22:00:00,"20230930-DK-Butterfly-1, Inc.",2766,Welcome to Bed Bath and Beyond's Third Quarter...
3,8.329564e+07,2010-01-07 15:30:00,"Constellation Brands, Inc.",3662,"Good morning. My name is Julianne, and I will ..."
4,8.327706e+07,2010-01-07 16:00:00,Lennar Corporation,3976,"Thank you for standing by, and welcome to Lenn..."
...,...,...,...,...,...
28233,1.904562e+09,2024-12-19 16:00:00,FactSet Research Systems Inc.,2585,"Good day, and thank you for standing by. Welco..."
28234,1.911714e+09,2024-12-19 16:00:00,Lennar Corporation,5361,Welcome to Lennar's Fourth Quarter Earnings Co...
28235,1.904730e+09,2024-12-19 22:00:00,"NIKE, Inc.",4496,"Good afternoon, everyone. Welcome to NIKE, Inc..."
28236,1.875427e+09,2024-12-19 22:30:00,FedEx Corporation,5256,"Good day, and welcome to the FedEx Fiscal Year..."


## Creating a list matching company name to ticker
There are a few lists online that match the tickers with the company name. One such website is the following:
- https://stockanalysis.com/stocks/

Since the matching is likely not perfect, I will need to manually match all the company names with their respective tickers. Furthermore, some of the S&P500 companies since 2010 will have gone bankrupt and not appear in the list. Therefore, I will choose to limit my data a bit more, only focusing on the transcript in 2015 and after (which still leaves me with 10 years of transcripts to analyze).

In [50]:
os.getcwd()

'/Users/danielwang/Desktop/Work stuff/Coding Stuff/Berkeley Github Stuff/MIDS-DATASCI-266-Natural-Language-Processing/MIDS 266 Final Project Predicting Post-Earnings Announcement Drift Using NLP'

In [5]:
# Starting by obtaining all the company names since 2015
company_list = merged_tscrpts[merged_tscrpts.Date >= pd.to_datetime('2015-01-01')]['Company_name'].unique()
company_list.sort()

len(company_list)

649

In [6]:
# Storing the list of the companies in a csv file
pd.Series(company_list).to_csv('./00_Data/SNP500_Company_Names_2015_to_2024.csv')

Next, I will create a new excel file with all available tickers, and I'll match it in Excel. The following workbook contains the results of the manual matching:
- SNP500_Company_Name_to_Ticker.xlsx

Afteward, I transferred just the names and tickers into a separate csv file, as shown below:

In [7]:
# Reading back the file
snp500_names_tickers = pd.read_csv('./00_Data/SNP500_Company_Names_Tickers_2015_to_2024.csv', index_col=0)
snp500_names_tickers.columns = ['Company_name', 'Ticker', 'Skip']

snp500_names_tickers

,Company_name,Ticker,Skip
0,"20230930-DK-Butterfly-1, Inc.",BBBY,NaN
1,3M Company,MMM,NaN
2,A. O. Smith Corporation,AOS,NaN
3,"AMETEK, Inc.",AME,NaN
4,"ANSYS, Inc.",ANSS,NaN
...,...,...,...
644,"Zimmer Biomet Holdings, Inc.",ZBH,NaN
645,"Zions Bancorporation, National Association",ZION,NaN
646,Zoetis Inc.,ZTS,NaN
647,eBay Inc.,EBAY,NaN


A "Y" flag in the Skip column indicates that the ticker should not be used, either because it's now used for a different company or because the original stock is traded under a different ticker (or parent ticker). I provide more detailed reasoning in the Excel file above, but these details can also be googled.

Now, we can first remove all tickers with a "Y" label in the Skip column. Then, we'll check for duplicate tickers (due to parent or subsidiary companies) and decide whether they need to be removed.

In [8]:
# Removing all tickers with 'Y' label in Skip column (determined manually)
snp500_names_tickers = snp500_names_tickers[snp500_names_tickers['Skip'] != 'Y']

# Checking for duplicates
duplicate_tickers = snp500_names_tickers['Ticker'].value_counts()
duplicate_tickers = list(duplicate_tickers[duplicate_tickers > 1].index)
duplicate_tickers_df = snp500_names_tickers[snp500_names_tickers['Ticker'].isin(duplicate_tickers)]

duplicate_tickers_df

,Company_name,Ticker,Skip
71,Baker Hughes Company,BKR,NaN
72,Baker Hughes Holdings LLC,BKR,NaN
90,Broadcom Corporation,AVGO,NaN
91,Broadcom Inc.,AVGO,NaN
133,Chubb Corporation,CB,NaN
134,Chubb Limited,CB,NaN
136,Cimarex Energy Co.,CTRA,NaN
160,Coterra Energy Inc.,CTRA,NaN
187,"Dr Pepper Snapple Group, Inc.",KDP,NaN
225,"FMC Technologies, Inc.",FTI,NaN


Again, due to acquisitions and such, we have these duplicated tickers. However we may not need to remove them, as shown below for BKR:

In [52]:
# Checking out BKR
display(merged_tscrpts[merged_tscrpts['Company_name'] == 'Baker Hughes Holdings LLC']['Date'])
display(merged_tscrpts[merged_tscrpts['Company_name'] == 'Baker Hughes Company']['Date'])

84      2010-01-26 15:00:00
771     2010-05-04 12:30:00
1225    2010-08-03 12:30:00
1685    2010-11-01 12:30:00
1928    2011-01-25 15:00:00
2472    2011-04-27 12:30:00
2900    2011-07-25 12:30:00
3540    2011-11-01 12:30:00
3774    2012-01-24 13:30:00
4297    2012-04-24 12:30:00
4749    2012-07-20 12:00:00
5218    2012-10-19 12:00:00
5667    2013-01-23 13:00:00
6147    2013-04-19 12:00:00
6617    2013-07-19 12:00:00
7083    2013-10-18 12:00:00
7520    2014-01-21 13:00:00
8013    2014-04-17 13:00:00
8475    2014-07-17 12:30:00
8952    2014-10-16 12:30:00
9420    2015-01-20 13:00:00
12354   2016-07-28 12:00:00
12727   2016-10-25 12:30:00
Name: Date, dtype: datetime64[ns]

14987   2018-01-24 14:30:00
15456   2018-04-20 13:30:00
15923   2018-07-20 13:30:00
16552   2018-10-30 13:00:00
16944   2019-01-31 13:30:00
17483   2019-04-30 13:30:00
17991   2019-07-31 13:30:00
18431   2019-10-30 13:00:00
18705   2020-01-22 14:00:00
19206   2020-04-22 13:00:00
19671   2020-07-22 13:00:00
20157   2020-10-21 13:00:00
20607   2021-01-21 14:00:00
21096   2021-04-21 12:00:00
21564   2021-07-21 12:30:00
22041   2021-10-20 12:30:00
22509   2022-01-20 13:30:00
22993   2022-04-20 12:30:00
23473   2022-07-20 12:30:00
23948   2022-10-19 12:30:00
24430   2023-01-23 13:30:00
24901   2023-04-19 12:30:00
25379   2023-07-19 12:30:00
26004   2023-10-26 13:30:00
26374   2024-01-24 14:30:00
26901   2024-04-24 13:30:00
27452   2024-07-26 13:30:00
27851   2024-10-23 13:30:00
Name: Date, dtype: datetime64[ns]

As shown above, "Baker Hughes Holdings LLC" was the old name used before they changed their name to "Baker Hughes Company". Therefore, let's check if the dates are separate for *all* of the tickers and their respective company names.

Before that though, let's first merge the company and ticker df with the transcripts.

## Getting one merged dataframe with transcripts and tickers
Merging the company and ticker dataframe with the transcript dataframe. Note that as mentioned above, we'll only be looking at transcripts 2015 and onward.

In [53]:
# Only getting transcripts since 2015
full_tscrpt_df = merged_tscrpts[merged_tscrpts['Date'] >= pd.to_datetime('2015-01-01')].copy()

# Merging the company and ticker df with the transcript df
full_tscrpt_df = pd.merge(full_tscrpt_df, snp500_names_tickers, on='Company_name', how='left')
full_tscrpt_df = full_tscrpt_df[full_tscrpt_df['Skip'] != 'Y']

# Dropping Unique_transcript_id and Skip
full_tscrpt_df = full_tscrpt_df.drop(columns=['Unique_transcript_id', 'Skip'], axis=1)

full_tscrpt_df

,Date,Company_name,Word_count,Text,Ticker
0,2015-01-06 21:30:00,"Micron Technology, Inc.",5184,"Good afternoon. My name is Karen, and I'll be ...",MU
1,2015-01-07 14:30:00,Monsanto Company,6631,"Greetings, and welcome to the First Quarter Fi...",MON
2,2015-01-08 15:30:00,"Constellation Brands, Inc.",3188,"Ladies and gentlemen, thank you for standing b...",STZ
3,2015-01-08 22:00:00,"20230930-DK-Butterfly-1, Inc.",4326,Welcome to the Bed Bath & Beyond's Third Quart...,BBBY
4,2015-01-12 22:00:00,Alcoa Inc.,7601,"Good day, ladies and gentlemen, and welcome to...",AA
...,...,...,...,...,...
18835,2024-12-19 16:00:00,FactSet Research Systems Inc.,2585,"Good day, and thank you for standing by. Welco...",FDS
18836,2024-12-19 16:00:00,Lennar Corporation,5361,Welcome to Lennar's Fourth Quarter Earnings Co...,LEN
18837,2024-12-19 22:00:00,"NIKE, Inc.",4496,"Good afternoon, everyone. Welcome to NIKE, Inc...",NKE
18838,2024-12-19 22:30:00,FedEx Corporation,5256,"Good day, and welcome to the FedEx Fiscal Year...",FDX


## Checking for separated dates by company name for duplicated tickers
Once again, for the duplicated tickers that we identified above, I'm going to confirm that the dates don't overlap (so even if we have two companies with seemingly the same ticker, they'll actually be the same company, just with a name change).

In [54]:
# Going through each group (by ticker) and checking if dates overlap
duplicate_company_names = list(duplicate_tickers_df['Company_name'])
duplicate_tscrpt_dict = {ticker: [] for ticker in duplicate_tickers}
dup_tickers_tscrpts = full_tscrpt_df[full_tscrpt_df['Company_name'].isin(duplicate_company_names)]

for ticker, group in dup_tickers_tscrpts.groupby('Ticker'):
    group_value_counts = group['Date'].value_counts()
    duplicate_tscrpt_dict[ticker] += list((group_value_counts[group_value_counts > 1]).index)

duplicate_tscrpt_dict

{'FTI': [],
 'WBD': [],
 'AVGO': [],
 'CB': [],
 'LHX': [],
 'CTRA': [],
 'KDP': [],
 'BKR': []}

As we can see, there were no tickers with overlapping dates; however, if we look at each group more closely, we can see that the group with WDB seemingly has multiple companies until they merged:

In [55]:
# Checking out WBD:
group

,Date,Company_name,Word_count,Text,Ticker
323,2015-02-11 15:30:00,"Warner Media, LLC",5766,Welcome to the Time Warner Inc. Fourth Quarter...,WBD
378,2015-02-19 13:30:00,"Warner Bros. Discovery, Inc.",5045,"Good day, ladies and gentlemen, and welcome to...",WBD
704,2015-04-29 14:30:00,"Warner Media, LLC",4505,"Hello, and welcome to the Time Warner Incorpor...",WBD
801,2015-05-05 12:30:00,"Warner Bros. Discovery, Inc.",4319,"Good day, ladies and gentlemen, and welcome to...",WBD
1288,2015-08-05 12:30:00,"Warner Bros. Discovery, Inc.",4888,"Good day, ladies and gentlemen, and welcome to...",WBD
1297,2015-08-05 14:30:00,"Warner Media, LLC",4076,Welcome to the Time Warner Incorporated Second...,WBD
1710,2015-11-03 13:30:00,"Warner Bros. Discovery, Inc.",2949,"Good day, ladies and gentlemen, and welcome to...",WBD
1740,2015-11-04 15:30:00,"Warner Media, LLC",4845,Welcome to the Time Warner Inc. Third Quarter ...,WBD
2149,2016-02-10 15:30:00,"Warner Media, LLC",5140,Welcome to the Time Warner Fourth Quarter 2015...,WBD
2207,2016-02-18 13:30:00,"Warner Bros. Discovery, Inc.",3950,"Good day, ladies and gentlemen, and welcome to...",WBD


Therefore, just to make sure we have the write reference ticker, I'll remove Warner Media, LLC from the dataset. Similarly, I'll manually check the rest of the tickers to confirm any others that I need to remove (again, the duplicated tickers likely came from mergers and acquisitions).

In [56]:
for ticker, group in dup_tickers_tscrpts.groupby('Ticker'):
    display(group)

,Date,Company_name,Word_count,Text,Ticker
173,2015-01-29 21:45:00,Broadcom Corporation,1950,Welcome to the Broadcom Fourth Quarter 2014 Ea...,AVGO
423,2015-02-25 22:00:00,Broadcom Inc.,2475,Welcome to the Avago Technologies Limited Firs...,AVGO
538,2015-04-21 20:45:00,Broadcom Corporation,1770,Welcome to the Broadcom Q1 2015 Earnings Call....,AVGO
1364,2015-08-26 21:00:00,Broadcom Inc.,2235,"Good day, ladies and gentlemen, and welcome to...",AVGO
1827,2015-12-02 22:00:00,Broadcom Inc.,2365,Welcome to the Avago Technologies Limited Four...,AVGO
2285,2016-03-03 22:00:00,Broadcom Inc.,2804,"Good day, ladies and gentlemen. Welcome to Bro...",AVGO
2748,2016-06-02 21:00:00,Broadcom Inc.,2758,Welcome to Broadcom Limited Second Quarter Fis...,AVGO
3208,2016-09-01 21:00:00,Broadcom Inc.,2160,Welcome to Broadcom Limited's Third Quarter Fi...,AVGO
3671,2016-12-08 22:00:00,Broadcom Inc.,2891,"Good day, ladies and gentlemen. Welcome to Bro...",AVGO
4120,2017-03-01 22:00:00,Broadcom Inc.,2105,Welcome to Broadcom Limited First Quarter Fisc...,AVGO


,Date,Company_name,Word_count,Text,Ticker
22,2015-01-20 13:00:00,Baker Hughes Holdings LLC,3993,"My name is Paulette, and I will be your confer...",BKR
2956,2016-07-28 12:00:00,Baker Hughes Holdings LLC,4387,"Good day, ladies and gentlemen, and welcome to...",BKR
3329,2016-10-25 12:30:00,Baker Hughes Holdings LLC,3729,"Good day, ladies and gentlemen, and welcome to...",BKR
5589,2018-01-24 14:30:00,Baker Hughes Company,4694,"Good day, ladies and gentlemen, and welcome to...",BKR
6058,2018-04-20 13:30:00,Baker Hughes Company,4334,"Good day, ladies and gentlemen, and welcome to...",BKR
6525,2018-07-20 13:30:00,Baker Hughes Company,4705,"Good day, ladies and gentlemen, and welcome to...",BKR
7154,2018-10-30 13:00:00,Baker Hughes Company,4933,"Good day, ladies and gentlemen, and welcome to...",BKR
7546,2019-01-31 13:30:00,Baker Hughes Company,4938,"Good day, ladies and gentlemen, and welcome to...",BKR
8085,2019-04-30 13:30:00,Baker Hughes Company,3800,"Good day, ladies and gentlemen, and welcome to...",BKR
8593,2019-07-31 13:30:00,Baker Hughes Company,4163,"Good day, ladies and gentlemen, and welcome to...",BKR


,Date,Company_name,Word_count,Text,Ticker
111,2015-01-28 13:30:00,Chubb Limited,2847,"Good day, and welcome to the ACE Limited Fourt...",CB
176,2015-01-29 22:00:00,Chubb Corporation,4191,"Good day, everyone, and welcome to the Chubb C...",CB
547,2015-04-22 12:30:00,Chubb Limited,2160,"Good day, and welcome to the ACE Limited First...",CB
615,2015-04-23 21:00:00,Chubb Corporation,3410,"Good day, everyone, and welcome to The Chubb C...",CB
998,2015-07-22 12:30:00,Chubb Limited,2023,"Good day, and welcome to ACE Limited's Second ...",CB
1461,2015-10-21 12:30:00,Chubb Limited,2317,"Good day, and welcome to the ACE Limited's Thi...",CB
1931,2016-01-27 13:30:00,Chubb Limited,2872,"Good day, and welcome to the Chubb Limited Fou...",CB
2651,2016-05-05 12:30:00,Chubb Limited,3073,"Good day, and welcome to the Chubb Limited Fir...",CB
2910,2016-07-27 12:30:00,Chubb Limited,2530,"Good day, and welcome to the Chubb Limited Sec...",CB
3368,2016-10-26 12:30:00,Chubb Limited,2615,"Good day, and welcome to the Chubb Limited Thi...",CB


,Date,Company_name,Word_count,Text,Ticker
373,2015-02-18 16:00:00,Cimarex Energy Co.,3786,"Hello, and welcome to the Cimarex Energy Fourt...",CTRA
394,2015-02-20 14:30:00,Coterra Energy Inc.,2351,"Good morning, and welcome to the Cabot Oil & G...",CTRA
630,2015-04-24 13:30:00,Coterra Energy Inc.,2174,"Good day, and welcome to the Cabot Oil & Gas C...",CTRA
812,2015-05-05 15:00:00,Cimarex Energy Co.,3450,"Hello, and welcome to the Cimarex Energy First...",CTRA
1081,2015-07-24 13:30:00,Coterra Energy Inc.,1984,"Good day, and welcome to the Cabot Oil & Gas C...",CTRA
1298,2015-08-05 15:00:00,Cimarex Energy Co.,4003,Welcome to the Cimarex Energy Second Quarter E...,CTRA
1527,2015-10-23 13:30:00,Coterra Energy Inc.,2684,"Good day, and welcome to the Cabot Oil & Gas C...",CTRA
1741,2015-11-04 16:00:00,Cimarex Energy Co.,3032,"Good morning, and welcome to the Cimarex Energ...",CTRA
2199,2016-02-17 16:00:00,Cimarex Energy Co.,3771,"Good day, and welcome to the Cimarex Energy Fo...",CTRA
2226,2016-02-19 14:30:00,Coterra Energy Inc.,2661,"Good morning, and welcome to the Cabot Oil & G...",CTRA


,Date,Company_name,Word_count,Text,Ticker
319,2015-02-11 14:00:00,"FMC Technologies, Inc.",2598,Welcome to the Fourth Quarter 2014 Earnings An...,FTI
552,2015-04-22 13:00:00,"FMC Technologies, Inc.",2080,"Good morning, and welcome to the First Quarter...",FTI
1004,2015-07-22 13:00:00,"FMC Technologies, Inc.",2202,Welcome to the Second Quarter 2015 earnings an...,FTI
1466,2015-10-21 13:00:00,"FMC Technologies, Inc.",2345,Welcome to the Q3 2015 Earnings Analyst Call. ...,FTI
2192,2016-02-17 14:00:00,"FMC Technologies, Inc.",3398,"Good morning. My name is Erica, and I will be ...",FTI
2466,2016-04-27 13:00:00,"FMC Technologies, Inc.",2899,"Good morning. My name is Robbie, and I'll be y...",FTI
2831,2016-07-21 13:00:00,"FMC Technologies, Inc.",3431,"Good morning. My name is Emily, and I will be ...",FTI
3292,2016-10-20 13:00:00,"FMC Technologies, Inc.",2872,"Good morning, my name is Kelly, and I will be ...",FTI
5916,2018-02-22 13:00:00,TechnipFMC plc,3110,"Good afternoon. My name is Kim, and I will be ...",FTI
6393,2018-05-10 12:00:00,TechnipFMC plc,3438,"Hello, and welcome to the TechnipFMC First Qua...",FTI


,Date,Company_name,Word_count,Text,Ticker
250,2015-02-04 22:00:00,Keurig Dr Pepper Inc.,4337,"Good afternoon, and welcome to the Keurig Gree...",KDP
344,2015-02-12 16:00:00,"Dr Pepper Snapple Group, Inc.",3247,"Good morning, and welcome to Dr Pepper Snapple...",KDP
604,2015-04-23 15:00:00,"Dr Pepper Snapple Group, Inc.",2606,"Good morning, and welcome to Dr Pepper Snapple...",KDP
838,2015-05-06 21:00:00,Keurig Dr Pepper Inc.,3059,"Good afternoon, and welcome to the Keurig Gree...",KDP
1053,2015-07-23 15:00:00,"Dr Pepper Snapple Group, Inc.",2899,"Good morning, and welcome to Dr Pepper Snapple...",KDP
1302,2015-08-05 21:00:00,Keurig Dr Pepper Inc.,3658,"Good afternoon, and welcome to the Keurig Gree...",KDP
1506,2015-10-22 15:00:00,"Dr Pepper Snapple Group, Inc.",2502,"Good morning, and welcome to Dr Pepper Snapple...",KDP
1804,2015-11-18 22:00:00,Keurig Dr Pepper Inc.,5302,"Good afternoon, and welcome to the Q4 and Year...",KDP
2201,2016-02-17 16:00:00,"Dr Pepper Snapple Group, Inc.",3222,"Good morning, and welcome to Dr Pepper Snapple...",KDP
2473,2016-04-27 14:00:00,"Dr Pepper Snapple Group, Inc.",2727,"Good morning, and welcome to Dr Pepper Snapple...",KDP


,Date,Company_name,Word_count,Text,Ticker
164,2015-01-29 16:00:00,"L3 Technologies, Inc.",3821,"Good day, and welcome to the L-3 Communication...",LHX
288,2015-02-06 13:30:00,"L3Harris Technologies, Inc.",3631,"Good day, ladies and gentlemen, and welcome to...",LHX
755,2015-04-30 15:00:00,"L3 Technologies, Inc.",3539,"Good day, and welcome to the L-3 Communication...",LHX
799,2015-05-05 12:30:00,"L3Harris Technologies, Inc.",2855,"Good day, ladies and gentlemen, and welcome to...",LHX
1206,2015-07-30 15:00:00,"L3 Technologies, Inc.",3658,"Good day, and welcome to the L-3 Communication...",LHX
1326,2015-08-07 12:30:00,"L3Harris Technologies, Inc.",2954,"Good day, ladies and gentlemen, and welcome to...",LHX
1651,2015-10-29 15:00:00,"L3 Technologies, Inc.",3551,"Good day, and welcome to the L-3 Communication...",LHX
1708,2015-11-03 13:30:00,"L3Harris Technologies, Inc.",3028,"Good day, ladies and gentlemen, and welcome to...",LHX
1985,2016-01-28 16:00:00,"L3 Technologies, Inc.",2813,"Good day, and welcome to the L-3 Communication...",LHX
2030,2016-02-02 13:30:00,"L3Harris Technologies, Inc.",2844,"Good day, ladies and gentlemen, and welcome to...",LHX


,Date,Company_name,Word_count,Text,Ticker
323,2015-02-11 15:30:00,"Warner Media, LLC",5766,Welcome to the Time Warner Inc. Fourth Quarter...,WBD
378,2015-02-19 13:30:00,"Warner Bros. Discovery, Inc.",5045,"Good day, ladies and gentlemen, and welcome to...",WBD
704,2015-04-29 14:30:00,"Warner Media, LLC",4505,"Hello, and welcome to the Time Warner Incorpor...",WBD
801,2015-05-05 12:30:00,"Warner Bros. Discovery, Inc.",4319,"Good day, ladies and gentlemen, and welcome to...",WBD
1288,2015-08-05 12:30:00,"Warner Bros. Discovery, Inc.",4888,"Good day, ladies and gentlemen, and welcome to...",WBD
1297,2015-08-05 14:30:00,"Warner Media, LLC",4076,Welcome to the Time Warner Incorporated Second...,WBD
1710,2015-11-03 13:30:00,"Warner Bros. Discovery, Inc.",2949,"Good day, ladies and gentlemen, and welcome to...",WBD
1740,2015-11-04 15:30:00,"Warner Media, LLC",4845,Welcome to the Time Warner Inc. Third Quarter ...,WBD
2149,2016-02-10 15:30:00,"Warner Media, LLC",5140,Welcome to the Time Warner Fourth Quarter 2015...,WBD
2207,2016-02-18 13:30:00,"Warner Bros. Discovery, Inc.",3950,"Good day, ladies and gentlemen, and welcome to...",WBD


Ultimately I'll decide to remove the following companies due to overlapping tickers (and keep the more recent names):
- Broadcom Corporation
- Chubb Corporation
- Cimarex Energy Co.
- Dr Pepper Snapple Group, Inc.
- L3 Technologies, Inc.
- Warner Media, LLC

The following companies have no overlapping dates, even with the same ticker (likely just indicating a name change):
- Baker Hughes Holdings LLC and Baker Hughes Company
- FMC Technologies, Inc. and TechnipFMC plc

In [57]:
# Removing certain companies names from the full dataset
list_comp_names_to_remove = ['Broadcom Corporation', 'Chubb Corporation', 'Cimarex Energy Co.', 'Dr Pepper Snapple Group, Inc.',
                             'L3 Technologies, Inc.', 'Warner Media, LLC']
full_tscrpt_df = full_tscrpt_df[~full_tscrpt_df['Company_name'].isin(list_comp_names_to_remove)]
full_tscrpt_df = full_tscrpt_df.reset_index(drop=True)

full_tscrpt_df

,Date,Company_name,Word_count,Text,Ticker
0,2015-01-06 21:30:00,"Micron Technology, Inc.",5184,"Good afternoon. My name is Karen, and I'll be ...",MU
1,2015-01-07 14:30:00,Monsanto Company,6631,"Greetings, and welcome to the First Quarter Fi...",MON
2,2015-01-08 15:30:00,"Constellation Brands, Inc.",3188,"Ladies and gentlemen, thank you for standing b...",STZ
3,2015-01-08 22:00:00,"20230930-DK-Butterfly-1, Inc.",4326,Welcome to the Bed Bath & Beyond's Third Quart...,BBBY
4,2015-01-12 22:00:00,Alcoa Inc.,7601,"Good day, ladies and gentlemen, and welcome to...",AA
...,...,...,...,...,...
18774,2024-12-19 16:00:00,FactSet Research Systems Inc.,2585,"Good day, and thank you for standing by. Welco...",FDS
18775,2024-12-19 16:00:00,Lennar Corporation,5361,Welcome to Lennar's Fourth Quarter Earnings Co...,LEN
18776,2024-12-19 22:00:00,"NIKE, Inc.",4496,"Good afternoon, everyone. Welcome to NIKE, Inc...",NKE
18777,2024-12-19 22:30:00,FedEx Corporation,5256,"Good day, and welcome to the FedEx Fiscal Year...",FDX


# Obtaining Stock Price Data
With my tickers uniquely specified, I can move onto obtaining the stock price data. This will be done using the yfinance API.

In [26]:
unique_tickers = list(full_tscrpt_df['Ticker'].unique())
problematic_tickers = ['BF.A', np.nan]
for ticker in problematic_tickers:
    unique_tickers.remove(ticker)
unique_tickers.sort()

In [28]:
# List of unique tickers (after removing some problematic tickers)
unique_tickers = list(full_tscrpt_df['Ticker'].unique())
problematic_tickers = ['BF.A', np.nan]
for ticker in problematic_tickers:
    unique_tickers.remove(ticker)
unique_tickers.sort()

# Set date range
start_date_analyzed = '2015-01-01'
end_date_analyzed = '2025-01-01'

# Getting data for S&P500 Stocks
yfinance_stock_data = yf.download(unique_tickers, start=start_date_analyzed, end=end_date_analyzed)

# NOTE: there will be missing stocks because Yahoo Finance has delisted some of the stocks that went out of the S&P500 (like SIVB)
yfinance_stock_data

/var/folders/35/1ybvyr4j2nv19s55hszmmqgh0000gn/T/ipykernel_74943/3709180698.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  yfinance_stock_data = yf.download(unique_tickers, start=start_date_analyzed, end=end_date_analyzed)
[*********************100%***********************]  627 of 627 completed

51 Failed downloads:
['VAR', 'FLIR', 'ETFC', 'WRK', 'TFCF', 'AIRC', 'CELG', 'RTN', 'CTLT', 'CERN', 'PBCT', 'RHT', 'ALXN', 'MON', 'BPYU', 'NBL', 'BBBY', 'KSU', 'TSS', 'XL', 'ABMD', 'DISH', 'WCG', 'CTXS', 'SWN', 'PDCO', 'TIF', 'ATVI', 'MRO', 'DO', 'AABA', 'PXD', 'MXIM', 'XLNX', 'CTA-A']: YFTzMissingError('possibly delisted; no timezone found')
['TWTR', 'LVLT', 'TYC', 'MJN', 'SNI', 'BCR', 'SPLS', 'MNSH', 'RAI', 'STJ', 'NLSN', 'LLTC', 'TWC', 'WFM', 'MNK']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-01)')
['SNDK']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-01) (Yahoo err

Price      Adj Close                                              ... Volume  \
Ticker          AABA ABMD AIRC ALXN ATVI BBBY BCR BPYU CELG CERN  ...   XLNX   
Date                                                              ...          
2015-01-02       NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  ...    NaN   
2015-01-05       NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  ...    NaN   
2015-01-06       NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  ...    NaN   
2015-01-07       NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  ...    NaN   
2015-01-08       NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  ...    NaN   
...              ...  ...  ...  ...  ...  ...  ..  ...  ...  ...  ...    ...   
2024-12-24       NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  ...    NaN   
2024-12-26       NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  ...    NaN   
2024-12-27       NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  ...    NaN   
2024-12-30       NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  ...    NaN   
2024-12-31       NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  ...    NaN   

Price                                                                      \
Ticker           XOM     XRAY      XRX      XYL      YUM      ZBH    ZBRA   
Date                                                                        
2015-01-02  10220400  1825800  1484111   605900  2283466   936579  411800   
2015-01-05  18502400  1469800  2668986  1369900  4418651  2223873  420300   
2015-01-06  16670700  1988400  2719345  1333200  5004401  1835563  527500   
2015-01-07  13590700   959800  1835414  1038600  4554134  1505860  467800   
2015-01-08  15487500  1409500  2363981   821800  4258268  1449004  324400   
...              ...      ...      ...      ...      ...      ...     ...   
2024-12-24   7807000   964900  2157000   379300   533000   458600   88700   
2024-12-26   9652400  2577100  2971100   575700  1040900  1277300  140100   
2024-12-27  11943900  1876900  1874700   552400  1146300   743400  287200   
2024-12-30  11080800  1965500  2120300   586800  1144600  1532000  211300   
2024-12-31  12387800  1595200  2035800   641600  1217100   683300  327900   

Price                         
Ticker         ZION      ZTS  
Date                          
2015-01-02  2298700  1784200  
2015-01-05  5326000  3112100  
2015-01-06  9086500  3977200  
2015-01-07  2759900  2481800  
2015-01-08  1831500  3121300  
...             ...      ...  
2024-12-24   371100  1023600  
2024-12-26  1158200  2167200  
2024-12-27   923400  1800100  
2024-12-30   827800  1531400  
2024-12-31  1426000  1327400  

[2516 rows x 3186 columns]

As expected, there are some tickers that have been delisted that we could not find. However, this might pose an issue with survivorship bias as there's a good chance the delisted stocks went out of business (or were acquired) soon after their earnings call. However, while this will likely affect the results of the analysis, this is the best we can do without paying for more data. Therefore, we'll continue with this project even knowing that survivorship bias is an issue.

With that being said, this doesn't take away from the fact that our models could still be a valid predictor of stock price movements. The primary thing it won't be able to do is identify *extreme* downturns.

## Data cleaning

In [30]:
# Dropping nulls based on column:
cleaned_stock_data = yfinance_stock_data.dropna(axis=1)

cleaned_stock_data

Price            Close                                                \
Ticker               A         AA        AAL         AAP        AAPL   
Date                                                                   
2015-01-02   37.195484  35.790691  51.079914  140.151520   24.288578   
2015-01-05   36.498528  33.717182  51.051491  138.304199   23.604334   
2015-01-06   35.929958  33.965111  50.255581  138.206924   23.606552   
2015-01-07   36.406830  34.844090  50.227150  141.176834   23.937571   
2015-01-08   37.498119  35.835770  50.843037  142.414307   24.857309   
...                ...        ...        ...         ...         ...   
2024-12-24  135.276016  38.355900  17.350000   42.970909  257.578674   
2024-12-26  135.007660  38.097610  17.350000   43.472057  258.396667   
2024-12-27  134.719391  37.432018  17.350000   43.295181  254.974930   
2024-12-30  133.606171  36.905510  17.620001   45.250637  251.593079   
2024-12-31  133.773468  37.531364  17.430000   46.469112  249.817368   

Price                                                                  ...  \
Ticker            ABBV         ABT       ACGL         ACN        ADBE  ...   
Date                                                                   ...   
2015-01-02   42.394871   36.580605  18.539352   74.623985   72.339996  ...   
2015-01-05   41.597023   36.588772  18.428413   73.364014   71.980003  ...   
2015-01-06   41.391125   36.173271  18.469618   72.834801   70.529999  ...   
2015-01-07   43.064026   36.466560  18.577387   74.363602   71.110001  ...   
2015-01-08   43.514404   37.216095  18.900694   75.497581   72.919998  ...   
...                ...         ...        ...         ...         ...  ...   
2024-12-24  175.172592  113.126747  92.669998  358.322815  447.940002  ...   
2024-12-26  174.394058  113.629478  92.930000  357.133789  450.160004  ...   
2024-12-27  173.235977  113.353477  92.339996  352.922638  446.480011  ...   
2024-12-30  171.474518  111.194641  91.889999  349.266388  445.799988  ...   
2024-12-31  172.934296  111.500229  92.349998  348.572815  444.679993  ...   

Price        Volume                                                         \
Ticker          XEL       XOM     XRAY      XRX      XYL      YUM      ZBH   
Date                                                                         
2015-01-02  2534900  10220400  1825800  1484111   605900  2283466   936579   
2015-01-05  3107200  18502400  1469800  2668986  1369900  4418651  2223873   
2015-01-06  4749600  16670700  1988400  2719345  1333200  5004401  1835563   
2015-01-07  2833400  13590700   959800  1835414  1038600  4554134  1505860   
2015-01-08  2516800  15487500  1409500  2363981   821800  4258268  1449004   
...             ...       ...      ...      ...      ...      ...      ...   
2024-12-24   943900   7807000   964900  2157000   379300   533000   458600   
2024-12-26  1394900   9652400  2577100  2971100   575700  1040900  1277300   
2024-12-27  2015000  11943900  1876900  1874700   552400  1146300   743400   
2024-12-30  2642900  11080800  1965500  2120300   586800  1144600  1532000   
2024-12-31  2143800  12387800  1595200  2035800   641600  1217100   683300   

Price                                 
Ticker        ZBRA     ZION      ZTS  
Date                                  
2015-01-02  411800  2298700  1784200  
2015-01-05  420300  5326000  3112100  
2015-01-06  527500  9086500  3977200  
2015-01-07  467800  2759900  2481800  
2015-01-08  324400  1831500  3121300  
...            ...      ...      ...  
2024-12-24   88700   371100  1023600  
2024-12-26  140100  1158200  2167200  
2024-12-27  287200   923400  1800100  
2024-12-30  211300   827800  1531400  
2024-12-31  327900  1426000  1327400  

[2516 rows x 2635 columns]

In [31]:
# Making cleaned_stock_data so that there are rows for each ticker for each date:
unstacked = cleaned_stock_data.stack(level=0).unstack(level=0)
unstacked = unstacked.transpose().reset_index()
unstacked.columns.name = None

unstacked

/var/folders/35/1ybvyr4j2nv19s55hszmmqgh0000gn/T/ipykernel_74943/3378961417.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  unstacked = cleaned_stock_data.stack(level=0).unstack(level=0)


,Ticker,Date,Close,High,Low,Open,Volume
0,A,2015-01-02,37.195484,37.883270,37.021243,37.764053,1529200.0
1,A,2015-01-05,36.498528,37.103779,36.406824,36.975393,2041800.0
2,A,2015-01-06,35.929958,36.700279,35.783231,36.507700,2080600.0
3,A,2015-01-07,36.406830,36.507706,36.030840,36.241761,3359700.0
4,A,2015-01-08,37.498119,37.580654,36.847016,36.902040,2116300.0
...,...,...,...,...,...,...,...
1325927,ZTS,2024-12-24,163.651352,163.989184,161.703840,162.657719,1023600.0
1325928,ZTS,2024-12-26,164.466125,164.734395,161.982043,162.677583,2167200.0
1325929,ZTS,2024-12-27,163.551987,165.469691,162.478861,163.899749,1800100.0
1325930,ZTS,2024-12-30,161.207016,163.005489,160.422037,162.846504,1531400.0


In [40]:
# Getting Date to datetime format and setting as index
unstacked['Date'] = pd.to_datetime(unstacked['Date'])
data = unstacked.sort_values(by='Ticker', ascending=True)
data = data.set_index('Date')

# Only keeping Open and Close prices since that's how we'll determine entry and exits 
data = data.drop(['High', 'Low', 'Volume'], axis=1)

# Re-ordering Open and Close columns
data = data[['Ticker', 'Open', 'Close']]

# Creating a new Date_col column to rank and drop dates appropriately
data = data.reset_index().set_index('Date', drop=False)
data = data.rename(columns={'Date': 'Date_col'})
data = data.sort_values(by=['Date_col', 'Ticker'])

data.head()

,Date_col,Ticker,Open,Close
Date,,,,
2015-01-02,2015-01-02,A,37.764053,37.195484
2015-01-02,2015-01-02,AA,35.768151,35.790691
2015-01-02,2015-01-02,AAL,51.430489,51.079914
2015-01-02,2015-01-02,AAP,142.175663,140.151520
2015-01-02,2015-01-02,AAPL,24.746224,24.288578


## Getting cleaned S&P500 data into a csv file
Doing this will allow us to not have constantly download data from yfinance each time (and we can get consistent data with consistent tickers).

In [43]:
# Getting the data into a csv file
data.to_csv('./00_Data/SP500_OC_data_2015_to_2024.csv')

## Deciding the outcome variable(s) we want to use
Since the goal is not to run an autoregressive model, my **target variable** will simply be **whether the stock price moved up or down at some time in the future compared to when we entered**. To keep things consistent, I will assume that I always enter at the next possible market open (9:30am ET), whether it's the same day as the earnings call, or the next business day after.

For example, if an earnings call happened on 10/08/2018 at 4pm, then - since I would enter at the market open the **following** day (10/09/2018) - I could pedict what would happen at perhaps the Close (4:00pm ET) price of that day, or potentially the Close price of the next day (10/10/2018), or the day after (10/11/2018), assuming they are all trading days. However, if an earnings call happened in the morning before the market opens (e.g., 6am ET on 10/08/2018), then - after entering at 9:30am ET on that **same** day - I could potentially predict the Close price of that day (10/08/2018), or the next day (10/09/2018), or even the one after that (10/10/2018).

Regardless of whether I enter the same day as the earnings call or the next business day, and regardless of how long I hold the stock, I will assume that I always exit a trade at the Close price on some particular day. Since my goal is to predict PEAD - while still making sure the drift effects are still due to the earnings call - I will start with 4 potential target variables (and narrow them down later). Specifically, I will predict the direction of the Close price - with respect to the Open price on the day I entered the trade - for the following:
1. The same day as I entered the trade (i.e., the direction change from 9:30am ET to 4:00pm ET that day)
2. One business day after my entry
3. Two business days after my entry
4. One week after my entry (or 5 business days after entry)

I will need to explore the data and my models more to narrow down which target variable I'll ultimately choose, but this provides some flexibility into how I want my model to work.

***Note**: I retroactively changed my data at this step to include the time as well. This is so I can find the correct Open and Close prices (i.e., whether to base my Close prices off the current day's Open price or the next business day's Open price - depending on what time the earnings call took place). I've made the changes above, but I kept my original analyses in the previous notebook.

In [63]:
os.getcwd()

'/Users/danielwang/Desktop/Work stuff/Coding Stuff/Berkeley Github Stuff/MIDS-DATASCI-266-Natural-Language-Processing/MIDS 266 Final Project Predicting Post-Earnings Announcement Drift Using NLP'

In [79]:
# Loading back in the data if necessary
# data = pd.read_csv('./00_Data/SP500_OC_data_2015_to_2024.csv')
# data['Date'] = pd.to_datetime(data['Date'])
# data['Date_col'] = pd.to_datetime(data['Date_col'])
# data = data.set_index('Date')
# data = data.sort_values(by=['Ticker', 'Date_col'])

# data

In [152]:
# Getting a copy of the data so I can get shifts in prices (along with sorting by ticker first, then date)
full_stock_data = data.copy()
full_stock_data = full_stock_data.sort_values(by=['Ticker', 'Date_col'])

# Getting shifted prices by 1 business day, 2 business days, and 5 business days (1 week)
full_stock_data['Close_1'] = full_stock_data.groupby('Ticker')['Close'].shift(-1)
full_stock_data['Close_2'] = full_stock_data.groupby('Ticker')['Close'].shift(-2)
full_stock_data['Close_5'] = full_stock_data.groupby('Ticker')['Close'].shift(-5)

# Removing the np.nan values and resetting index
full_stock_data = full_stock_data.dropna()
full_stock_data = full_stock_data.reset_index(drop=True)

full_stock_data

,Date_col,Ticker,Open,Close,Close_1,Close_2,Close_5
0,2015-01-02,A,37.764053,37.195484,36.498528,35.929958,37.222988
1,2015-01-05,A,36.975393,36.498528,35.929958,36.406830,36.782822
2,2015-01-06,A,36.507700,35.929958,36.406830,37.498119,36.269264
3,2015-01-07,A,36.241761,36.406830,37.498119,37.222988,35.819931
4,2015-01-08,A,36.902040,37.498119,37.222988,36.782822,34.857018
...,...,...,...,...,...,...,...
1323292,2024-12-17,ZTS,173.885762,169.702560,165.946640,162.478867,163.651352
1323293,2024-12-18,ZTS,168.987170,165.946640,162.478867,163.790466,164.466125
1323294,2024-12-19,ZTS,165.529312,162.478867,163.790466,163.234024,163.551987
1323295,2024-12-20,ZTS,162.717341,163.790466,163.234024,163.651352,161.207016


Now we nearly have the target variables that we can test with. Note that by shifting the Close prices, we technically lost some data past 2024-12-23; however, our last earnings call occurred on 2024-12-20, so we can still measure its effects.

The last thing we need to do before joining the data is converting the actual prices to up or down effects. I'll keep the actual prices in case we want to do PnL analyses later, but what we really want to predict is the direction of movement based on the earnings call. To simplify things, a value of 1 will mean that the stock price went up since entry, and a value of 0 will mean that the stock price went down since entry (or stayed the same).

In [153]:
# Getting the price directions (where 0 is assumed to be negative)
full_stock_data['Close_0_dir'] = full_stock_data['Close'] - full_stock_data['Open']
full_stock_data['Close_1_dir'] = full_stock_data['Close_1'] - full_stock_data['Open']
full_stock_data['Close_2_dir'] = full_stock_data['Close_2'] - full_stock_data['Open']
full_stock_data['Close_5_dir'] = full_stock_data['Close_5'] - full_stock_data['Open']

full_stock_data['Close_0_dir'] = np.where(full_stock_data['Close_0_dir'] > 0, 1, 0)
full_stock_data['Close_1_dir'] = np.where(full_stock_data['Close_1_dir'] > 0, 1, 0)
full_stock_data['Close_2_dir'] = np.where(full_stock_data['Close_2_dir'] > 0, 1, 0)
full_stock_data['Close_5_dir'] = np.where(full_stock_data['Close_5_dir'] > 0, 1, 0)

full_stock_data

,Date_col,Ticker,Open,Close,Close_1,Close_2,Close_5,Close_0_dir,Close_1_dir,Close_2_dir,Close_5_dir
0,2015-01-02,A,37.764053,37.195484,36.498528,35.929958,37.222988,0,0,0,0
1,2015-01-05,A,36.975393,36.498528,35.929958,36.406830,36.782822,0,0,0,0
2,2015-01-06,A,36.507700,35.929958,36.406830,37.498119,36.269264,0,0,1,0
3,2015-01-07,A,36.241761,36.406830,37.498119,37.222988,35.819931,1,1,1,0
4,2015-01-08,A,36.902040,37.498119,37.222988,36.782822,34.857018,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1323292,2024-12-17,ZTS,173.885762,169.702560,165.946640,162.478867,163.651352,0,0,0,0
1323293,2024-12-18,ZTS,168.987170,165.946640,162.478867,163.790466,164.466125,0,0,0,0
1323294,2024-12-19,ZTS,165.529312,162.478867,163.790466,163.234024,163.551987,0,0,0,0
1323295,2024-12-20,ZTS,162.717341,163.790466,163.234024,163.651352,161.207016,1,1,1,0


# Merging the Transcripts with the Outcome Variables
Finally, we can merge the transcripts with our target variables (which, again, we can narrow down later). Before we do that, however, we need to set which date to match to. Specifically, if the earnings call took place anytime after 9:30am, our entry will be the next business day. However, if the earnings call took place before 9:30am, our entry will be the same business day.

## Checking the times of earnings calls
This is so we can get appropriate entries.

In [120]:
# Getting the unique set of datetimes and getting the value_counts of those datetimes
dts = full_tscrpt_df['Date'].dt.time.unique()
dts = np.sort(dts)
display(dts)
display(full_tscrpt_df[full_tscrpt_df['Date'].dt.time < pd.to_datetime('2000-01-01 09:30:00').time()])

array([datetime.time(0, 0), datetime.time(4, 30), datetime.time(11, 0),
       datetime.time(11, 30), datetime.time(11, 45), datetime.time(12, 0),
       datetime.time(12, 1), datetime.time(12, 15), datetime.time(12, 30),
       datetime.time(12, 31), datetime.time(12, 45), datetime.time(13, 0),
       datetime.time(13, 1), datetime.time(13, 15), datetime.time(13, 30),
       datetime.time(13, 45), datetime.time(14, 0), datetime.time(14, 1),
       datetime.time(14, 15), datetime.time(14, 30), datetime.time(15, 0),
       datetime.time(15, 1), datetime.time(15, 30), datetime.time(16, 0),
       datetime.time(16, 1), datetime.time(16, 30), datetime.time(17, 0),
       datetime.time(17, 1), datetime.time(17, 30), datetime.time(18, 0),
       datetime.time(18, 30), datetime.time(19, 0), datetime.time(19, 30),
       datetime.time(20, 0), datetime.time(20, 15), datetime.time(20, 30),
       datetime.time(20, 31), datetime.time(20, 45), datetime.time(21, 0),
       datetime.time(21, 1), dat

,Date,Company_name,Word_count,Text,Ticker
178,2015-01-30,"Franklin Resources, Inc.",2136,Welcome to Franklin Resources Earnings Comment...,BEN
370,2015-02-19,Walmart Inc.,12701,"Hello. This is Carol Schumacher, Vice Presiden...",WMT
673,2015-04-29,"Franklin Resources, Inc.",2437,Welcome to Franklin Resources earnings comment...,BEN
871,2015-05-19,Walmart Inc.,8995,"Welcome. This is Carol Shumacher, Vice Preside...",WMT
1116,2015-07-29,"Franklin Resources, Inc.",2140,Welcome to Franklin Resources Earnings Comment...,BEN
...,...,...,...,...,...
18498,2024-10-30,The Kraft Heinz Company,3991,"Hello. This is Anne-Marie Megela, Head of Glob...",KHC
18619,2024-11-04,Loews Corporation,2180,[ The transcript was presubmitted by Loews Cor...,L
18738,2024-11-26,The J. M. Smucker Company,4259,"Thank you, Crystal, and good morning, everyone...",SJM
18762,2024-12-18,"General Mills, Inc.",3791,"Good morning. This is Jeff Siemon, Vice Presid...",GIS


We notice that there are an extremely few number of earnings calls that happen before 9:30am ET, which was definitely a surprise to me. However, if we look at one of them more closely (the AES Corporation earnings call on 2023-08-04)...

In [122]:
full_tscrpt_df[full_tscrpt_df['Date'].dt.time == pd.to_datetime('2000-01-01 04:30:00').time()]

,Date,Company_name,Word_count,Text,Ticker
16280,2023-08-04 04:30:00,The AES Corporation,1206,"Good morning, everyone, and welcome to today's...",AES


...a quick Google Search will tell us that that earnings call in reality took place at 10am ET. Therefore, it appears some of the data is misleading. Furthermore, we notice that the remaining datetimes (before 9:30am ET) are at midnight, which seems unlikely. One way to resolve this is to manually go through every misleading earnings call date to see if I can find a resolution, but a more practical method would be to assume that that entry was simply not filled out. Due to the sheer number of earnings calls that occur after the market opens, I will assume that these remaining earnings calls all happened sometime after 9:30am ET.

Therefore, for *all* of the earnings calls, I will assume that I enter into the trade at 9:30am ET the **following day**, which will subsequently allow me to appropriately merge my stock price dataframe with this one.

*Note: For any columns with NaN stock price data, that simply means that I do not have access to it, and I will remove that data (and their respective transcripts) from my analysis.

## Shifting my stock price data forward by one date, then merging
To do this without having to use pd.merge_asof(), I can simply shift all my stock price data over by one date (or, more easily, shifting the Date_col back by one date).

In [154]:
# Shifting Date_col back by one date
full_stock_data['Shifted_date'] = full_stock_data.groupby('Ticker')['Date_col'].shift(1)
full_stock_data = full_stock_data.dropna()
full_stock_data = full_stock_data.reset_index(drop=True)

full_stock_data.head()

,Date_col,Ticker,Open,Close,Close_1,Close_2,Close_5,Close_0_dir,Close_1_dir,Close_2_dir,Close_5_dir,Shifted_date
0,2015-01-05,A,36.975393,36.498528,35.929958,36.406830,36.782822,0,0,0,0,2015-01-02
1,2015-01-06,A,36.507700,35.929958,36.406830,37.498119,36.269264,0,0,1,0,2015-01-05
2,2015-01-07,A,36.241761,36.406830,37.498119,37.222988,35.819931,1,1,1,0,2015-01-06
3,2015-01-08,A,36.902040,37.498119,37.222988,36.782822,34.857018,1,1,0,0,2015-01-07
4,2015-01-09,A,37.598978,37.222988,36.782822,36.269264,35.077118,0,0,0,0,2015-01-08


In [164]:
# Getting just the date as a column in full_tscrpt_df
full_tscrpt_df['Date_only'] = pd.to_datetime(full_tscrpt_df['Date'].dt.date)

# Merging my transcript data with the stock price data to get the target variables
full_combined_data = pd.merge(full_tscrpt_df, full_stock_data, left_on=['Ticker', 'Date_only'], right_on=['Ticker', 'Shifted_date'],
                              how='left')

# Dropping NaN rows (for missing stock data) and extraneous date columns, renaming Date_col to Next_biz_day, and reordering columns
full_combined_data = full_combined_data.dropna()
full_combined_data = full_combined_data.drop(columns=['Date_only', 'Shifted_date'], axis=1)
full_combined_data = full_combined_data.rename(columns={'Date_col': 'Next_biz_day'})
new_column_order = ['Date', 'Next_biz_day', 'Company_name', 'Ticker', 'Word_count', 'Text', 'Open', 'Close', 'Close_1', 'Close_2',
                   'Close_5', 'Close_0_dir', 'Close_1_dir', 'Close_2_dir', 'Close_5_dir']
full_combined_data = full_combined_data[new_column_order]
full_combined_data = full_combined_data.reset_index(drop=True)

full_combined_data

,Date,Next_biz_day,Company_name,Ticker,Word_count,Text,Open,Close,Close_1,Close_2,Close_5,Close_0_dir,Close_1_dir,Close_2_dir,Close_5_dir
0,2015-01-06 21:30:00,2015-01-07,"Micron Technology, Inc.",MU,5184,"Good afternoon. My name is Karen, and I'll be ...",30.795428,31.352146,32.885567,32.748825,29.349901,1.0,1.0,1.0,0.0
1,2015-01-08 15:30:00,2015-01-09,"Constellation Brands, Inc.",STZ,3188,"Ladies and gentlemen, thank you for standing b...",92.797200,93.153008,92.831917,92.250504,96.849953,1.0,1.0,0.0,1.0
2,2015-01-12 22:00:00,2015-01-13,Alcoa Inc.,AA,7601,"Good day, ladies and gentlemen, and welcome to...",37.120452,35.610390,33.694641,33.739716,35.880848,0.0,0.0,0.0,0.0
3,2015-01-14 13:30:00,2015-01-15,CSX Corporation,CSX,1909,"Good morning, ladies and gentlemen, and welcom...",9.566863,9.651751,9.863967,9.773422,9.753616,1.0,1.0,1.0,1.0
4,2015-01-14 13:30:00,2015-01-15,JPMorgan Chase & Co.,JPM,2758,"Good morning, ladies and gentlemen, and welcom...",41.869368,41.313416,42.019623,41.854340,42.583084,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17228,2024-12-19 16:00:00,2024-12-20,FactSet Research Systems Inc.,FDS,2585,"Good day, and thank you for standing by. Welco...",482.339360,481.274323,487.067291,489.067993,480.438232,0.0,1.0,1.0,0.0
17229,2024-12-19 16:00:00,2024-12-20,Lennar Corporation,LEN,5361,Welcome to Lennar's Fourth Quarter Earnings Co...,131.804181,131.985794,131.909332,131.909332,130.188751,1.0,1.0,1.0,0.0
17230,2024-12-19 22:00:00,2024-12-20,"NIKE, Inc.",NKE,4496,"Good afternoon, everyone. Welcome to NIKE, Inc...",75.078526,76.047157,75.869247,75.898895,73.783737,1.0,1.0,1.0,0.0
17231,2024-12-19 22:30:00,2024-12-20,FedEx Corporation,FDX,5256,"Good day, and welcome to the FedEx Fiscal Year...",290.526486,272.472351,265.831726,268.628265,277.235382,0.0,0.0,0.0,0.0


In [166]:
# Checking how many tickers I have in total
full_combined_data.Ticker.nunique()

527

## Saving the final combined data
Now that we have our transcripts, target variables, and prices (if we need them later), we can download this dataset to get started with our modeling. Once again, because of the size of the data, I will save it to my local computer. However, recreating it simply requires going through the steps in these notebooks.

In [167]:
# Saving the data locally
full_data_path = '/Users/danielwang/Desktop/Berkeley MIDS Stuff/Berkeley MIDS Summer 2025 Stuff/Berkeley MIDS DATASCI 266/Berkeley MIDS DATASCI 266 Project Material/SNP500_Transcripts_Price_2010_to_2024.csv'
full_combined_data.to_csv(full_data_path, sep='|')

In [170]:
# Retrieving just to double check it can load
tscrpt_price_df = pd.read_csv(full_data_path, sep='|', index_col=0)

tscrpt_price_df

,Date,Next_biz_day,Company_name,Ticker,Word_count,Text,Open,Close,Close_1,Close_2,Close_5,Close_0_dir,Close_1_dir,Close_2_dir,Close_5_dir
0,2015-01-06 21:30:00,2015-01-07,"Micron Technology, Inc.",MU,5184,"Good afternoon. My name is Karen, and I'll be ...",30.795428,31.352146,32.885567,32.748825,29.349901,1.0,1.0,1.0,0.0
1,2015-01-08 15:30:00,2015-01-09,"Constellation Brands, Inc.",STZ,3188,"Ladies and gentlemen, thank you for standing b...",92.797200,93.153008,92.831917,92.250504,96.849953,1.0,1.0,0.0,1.0
2,2015-01-12 22:00:00,2015-01-13,Alcoa Inc.,AA,7601,"Good day, ladies and gentlemen, and welcome to...",37.120452,35.610390,33.694641,33.739716,35.880848,0.0,0.0,0.0,0.0
3,2015-01-14 13:30:00,2015-01-15,CSX Corporation,CSX,1909,"Good morning, ladies and gentlemen, and welcom...",9.566863,9.651751,9.863967,9.773422,9.753616,1.0,1.0,1.0,1.0
4,2015-01-14 13:30:00,2015-01-15,JPMorgan Chase & Co.,JPM,2758,"Good morning, ladies and gentlemen, and welcom...",41.869368,41.313416,42.019623,41.854340,42.583084,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17228,2024-12-19 16:00:00,2024-12-20,FactSet Research Systems Inc.,FDS,2585,"Good day, and thank you for standing by. Welco...",482.339360,481.274323,487.067291,489.067993,480.438232,0.0,1.0,1.0,0.0
17229,2024-12-19 16:00:00,2024-12-20,Lennar Corporation,LEN,5361,Welcome to Lennar's Fourth Quarter Earnings Co...,131.804181,131.985794,131.909332,131.909332,130.188751,1.0,1.0,1.0,0.0
17230,2024-12-19 22:00:00,2024-12-20,"NIKE, Inc.",NKE,4496,"Good afternoon, everyone. Welcome to NIKE, Inc...",75.078526,76.047157,75.869247,75.898895,73.783737,1.0,1.0,1.0,0.0
17231,2024-12-19 22:30:00,2024-12-20,FedEx Corporation,FDX,5256,"Good day, and welcome to the FedEx Fiscal Year...",290.526486,272.472351,265.831726,268.628265,277.235382,0.0,0.0,0.0,0.0
